# [DEPRECATED] Updated Mask R-CNN (Matterport) - Train cell nucleus Dataset
This is an updated version of [Mask R-CNN - Train cell nucleus Dataset](https://colab.research.google.com/github/navidyou/Mask-RCNN-implementation-for-cell-nucleus-detection-executable-on-google-colab-/blob/master/mask_RCNN_cell_nucleus_google_colab.ipynb) for Google Colab. 

## Initialisation


Here are the main variables to customize the training. Be sure to set the number of epoch you want along with the correct weights file to use as a base.

In [ ]:
# Up to which epoch it should train
NB_EPOCHS = 100

# To restart or continue training, set to last
# If custom is choosed, please set custom_weights_file to true name of the file (should be in ./logs/ directory)
init_with = "base" #@param ["coco", "imagenet", "last", "base", "custom"]
custom_weights_file = "mask_rcnn_nephrologie_XXXX_XXX.h5"

# Format of images in the dataset (if wrapper used, it should be png)
IMAGE_FORMAT = 'jpg' #@param ['jp2', 'png', 'jpg', 'bmp']
# Side size of a division
DIVISION_SIZE = 1024

# For main training, set cortexMode to false
# Remaining of test about cortex detection, should be deleted or replaced
cortexMode = False 

In [ ]:
import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    import os
    import sys
    import random
    import math
    import re
    import time
    import numpy as np
    import cv2
    import matplotlib
    import matplotlib.pyplot as plt
    import json
    #import pandas as pd
    from shlex import quote
    from time import time, ctime
    from common_utils import progressBar, formatTime

    TOAST = True
    if TOAST:
        try:
            from win10toast import ToastNotifier
            toaster = ToastNotifier()
        except ModuleNotFoundError:
            TOAST = False

    from skimage.io import imread, imshow, imread_collection, concatenate_images
    from skimage.transform import resize
    
    classesInfo = [
        {"inferenceID": 1,  "id": 0,  "name": "cortex",            "color": "#ffaa00", "ignore": not cortexMode},
        {"inferenceID": 2,  "id": 1,  "name": "medullaire",        "color": "#ff0000", "ignore": not cortexMode},
        {"inferenceID": 3,  "id": 2,  "name": "fond",              "color": "#ffffff", "ignore": not cortexMode},
        {"inferenceID": 1,  "id": 3,  "name": "tubule_sain",       "color": "#ff007f", "ignore": cortexMode},
        {"inferenceID": 2,  "id": 4,  "name": "tubule_atrophique", "color": "#55557f", "ignore": cortexMode},
        {"inferenceID": 3,  "id": 5,  "name": "nsg",               "color": "#55007f", "ignore": cortexMode},
        {"inferenceID": 4,  "id": 6,  "name": "nsg_complet",       "color": "#ff557f", "ignore": cortexMode},
        {"inferenceID": 5,  "id": 7,  "name": "nsg_partiel",       "color": "#55aa7f", "ignore": cortexMode},
        {"inferenceID": 6,  "id": 8,  "name": "pac",               "color": "#ffaa7f", "ignore": cortexMode},
        {"inferenceID": 7,  "id": 9,  "name": "veine",             "color": "#0000ff", "ignore": cortexMode},
        {"inferenceID": 8,  "id": 10, "name": "vaisseau",          "color": "#55ff7f", "ignore": cortexMode},
        {"inferenceID": 9,  "id": 11, "name": "intima",            "color": "#aa0000", "ignore": cortexMode},
        {"inferenceID": 10, "id": 12, "name": "media",             "color": "#aa5500", "ignore": cortexMode}
    ]
    
    # Data Path
    if cortexMode:
        TRAIN_PATH = 'nephrology_cortex_dataset_train/'
        MAP_PATH = 'nephrology_cortex_dataset_val/'
    else:
        TRAIN_PATH = 'nephrology_dataset_train/'
        MAP_PATH = 'nephrology_dataset_val/'
    # Get train and test IDs
    train_ids = next(os.walk(TRAIN_PATH))[1]
    map_ids = next(os.walk(MAP_PATH))[1]

    NB_STEPS = len(train_ids)
    
    CUSTOM_CLASS_NAMES = []
    for classInfo in classesInfo:
        if not classInfo["ignore"]:
            CUSTOM_CLASS_NAMES.append(classInfo["name"])
    NB_CLASS = len(CUSTOM_CLASS_NAMES)
    COLOR_PER_CLASS = True
    evaluation_size = len(map_ids)
    # evaluation_size = 30 #@param {type:"slider", min:10, max:65, step:1}


    from mrcnn import config
    from mrcnn import utils
    from mrcnn import model
    from mrcnn import visualize

    from mrcnn.config import Config
    from mrcnn import utils
    from mrcnn import model as modellib
    from mrcnn import visualize
    from mrcnn.model import log

    %matplotlib inline 

    # Root directory of the project
    ROOT_DIR = os.getcwd()

    # Directories to save logs and trained model
    MODEL_DIR = os.path.join(ROOT_DIR, "logs")
    IMAGES_DIR = os.path.join(MODEL_DIR, "images")
    CONF_MATRIX_DIR = os.path.join(MODEL_DIR, "confusion_matrix")
    for path in [MODEL_DIR, IMAGES_DIR, CONF_MATRIX_DIR]:
        if not os.path.exists(path):
            os.makedirs(path)
    RESULT_CSV_PATH = os.path.join(MODEL_DIR, "auto_results.csv")
    if not os.path.exists(RESULT_CSV_PATH):
        with open(RESULT_CSV_PATH, 'w') as csv:
            csv.write("datetime; input_weights; output_weights; nb_steps; nb_epochs; duration; mAP")

    # Local path to trained weights file
    COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
    # Download COCO trained weights from Releases if needed
    if not os.path.exists(COCO_MODEL_PATH):
        utils.download_trained_weights(COCO_MODEL_PATH)

    print("Cell done")

## Configurations

In [ ]:
class CustomConfig(Config):
    """Configuration for training on the dataset.
    Derives from the base Config class and overrides values specific
    to the dataset.
    """
    # Give the configuration a recognizable name
    NAME = "skinet"

    # Train on 1 GPU and 1 images per GPU. We can put multiple images on each
    # GPU. Batch size is (GPUs * images/GPU).
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

    # Number of classes (including background)
    NUM_CLASSES = 1 + NB_CLASS  # background + all classes

    # Use small images for faster training. Set the limits of the small side
    # the large side, and that determines the image shape.
    IMAGE_MIN_DIM = DIVISION_SIZE
    IMAGE_MAX_DIM = DIVISION_SIZE

    # Use smaller anchors because our image and objects are small
    RPN_ANCHOR_SCALES = (8, 16, 64, 128, 256)  # anchor side in pixels

    # Aim to allow ROI sampling to pick 33% positive ROIs.
    TRAIN_ROIS_PER_IMAGE = 800

    # set number of epoch
    STEPS_PER_EPOCH = NB_STEPS // IMAGES_PER_GPU

    # set validation steps
    VALIDATION_STEPS = 50


config = CustomConfig()
config.display()

## Notebook Preferences

In [ ]:
def get_ax(rows=1, cols=1, size=8):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Change the default size attribute to control the size
    of rendered images
    """
    #fig, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    #return ax
    return plt.subplots(rows, cols, figsize=(size*cols, size*rows), frameon=False)

## Dataset

Create a synthetic dataset

Extend the Dataset class and add a method to load the shapes dataset, `load_shapes()`, and override the following methods:

* load_image()
* load_mask()
* image_reference()

In [ ]:
class CustomDataset(utils.Dataset):

    def __init__(self, dataset_id, custom_class_names, dataset_path, enable_occlusions=True):
        super().__init__()
        self.__ID = dataset_id
        self.__CUSTOM_CLASS_NAMES = custom_class_names.copy()
        self.__PATH = os.path.normpath(dataset_path)
        self.__ENABLE_OCCLUSIONS = enable_occlusions

    def get_class_names(self):
        return self.__CUSTOM_CLASS_NAMES.copy()

    def load_images(self):
        # Add classes
        for class_id, class_name in enumerate(self.__CUSTOM_CLASS_NAMES):
            self.add_class(self.__ID, class_id + 1, class_name)

        for id_ in os.listdir(self.__PATH):
            img_path = os.path.join(self.__PATH, id_, "images")
            img_path = os.path.join(img_path, os.listdir(img_path)[0])
            self.add_image(self.__ID, image_id=id_, path=img_path)

    def load_image(self, image_id):
        img = super().load_image(image_id)
        img = resize(img, (config.IMAGE_SHAPE[0], config.IMAGE_SHAPE[1]), mode='constant', preserve_range=True)
        return img

    def image_reference(self, image_id):
        """Return the data of the image."""
        info = self.image_info[image_id]
        if info["source"] == self.__ID:
            return info[self.__ID]
        else:
            super(self.__class__).image_reference(self, image_id)

    def load_mask(self, image_id):
        """Generate instance masks for cells of the given image ID.
        """
        info = self.image_info[image_id]
        info = info.get("id")

        path = os.path.join(self.__PATH, info)

        # Counting masks for current image
        number_of_masks = 0
        for masks_dir in os.listdir(path):
            # For each directory excepting /images
            if masks_dir not in self.__CUSTOM_CLASS_NAMES:
                continue
            masks_dir_path = os.path.join(path, masks_dir)
            # Adding length of directory https://stackoverflow.com/questions/2632205/how-to-count-the-number-of-files-in-a-directory-using-python
            number_of_masks += len([name_ for name_ in os.listdir(masks_dir_path) if os.path.isfile(os.path.join(masks_dir_path, name_))])

        # Reading masks
        masks = np.zeros([config.IMAGE_SHAPE[0], config.IMAGE_SHAPE[1], number_of_masks], dtype=np.uint8)
        classes = np.zeros((number_of_masks,), dtype=int)
        idx = 0
        for masks_dir in os.listdir(path):
            if masks_dir not in self.__CUSTOM_CLASS_NAMES:
                continue
            temp_class_id = self.__CUSTOM_CLASS_NAMES.index(masks_dir) + 1
            masks_dir_path = os.path.join(path, masks_dir)
            for mask_file in os.listdir(masks_dir_path):
                mask_ = imread(os.path.join(masks_dir_path, mask_file))
                mask_ = resize(mask_, (config.IMAGE_SHAPE[0], config.IMAGE_SHAPE[1]), mode='constant', preserve_range=True)
                masks[:, :, idx] = mask_
                classes[idx] = temp_class_id
                idx += 1
        # Handle occlusions
        if self.__ENABLE_OCCLUSIONS:
            occlusion = np.logical_not(masks[:, :, -1]).astype(np.uint8)
            for i in range(number_of_masks - 2, -1, -1):
                masks[:, :, i] = masks[:, :, i] * occlusion
                occlusion = np.logical_and(occlusion, np.logical_not(masks[:, :, i]))
        return masks, classes.astype(np.int32)

In [ ]:
# Training dataset
dataset_train = CustomDataset(config.NAME, CUSTOM_CLASS_NAMES, TRAIN_PATH, enable_occlusions=False)
dataset_train.load_images()
dataset_train.prepare()

# Validation dataset
dataset_val = CustomDataset(config.NAME, CUSTOM_CLASS_NAMES, MAP_PATH, enable_occlusions=False)
dataset_val.load_images()
dataset_val.prepare()

In [ ]:
# Load and display random samples
image_ids = np.random.choice(dataset_train.image_ids, 4)
for image_id in image_ids:
    image = dataset_train.load_image(image_id)
    mask, class_ids = dataset_train.load_mask(image_id)
    visualize.display_top_masks(image, mask, class_ids, dataset_train.class_names)

## Ceate Model

In [ ]:
# Create model in training mode
model = modellib.MaskRCNN(mode="training", config=config, model_dir=MODEL_DIR)

In [ ]:
# Which weights to start with?

if init_with == "imagenet":
    model.load_weights(model.get_imagenet_weights(), by_name=True)
elif init_with == "coco":
    # Load weights trained on MS COCO, but skip layers that
    # are different due to the different number of classes
    # See README for instructions to download the COCO weights
    model.load_weights(COCO_MODEL_PATH, by_name=True, exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", "mrcnn_bbox", "mrcnn_mask"])
elif init_with == "last":
    # Load the last model you trained and continue training
    listDir = os.listdir(MODEL_DIR)
    listDir.sort()
    lastDir = ''
    for logDir in listDir:
        if os.path.isdir(os.path.join(MODEL_DIR, logDir)) and 'skinet20' in logDir:
            lastDir = logDir
    lastDirPath = os.path.join(MODEL_DIR, lastDir)
    listDir = os.listdir(lastDirPath)
    listDir.sort()
    lastFile = ''
    for logFile in listDir:
        if '.h5' in logFile:
            lastFile = logFile
    if lastFile == '':
        print("Last file not found, using base")
        init_with = "base"
        weights_path = os.path.join(MODEL_DIR, 'mask_rcnn_base.h5')
    else:
        weights_path = os.path.join(lastDirPath, lastFile)
        print("Last file is " + weights_path)
    model.load_weights(weights_path, by_name=True,
                       exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", "mrcnn_bbox", "mrcnn_mask"])
elif init_with == "base":
    weights_path = os.path.join(MODEL_DIR, 'mask_rcnn_base.h5')
    model.load_weights(weights_path, by_name=True, exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", "mrcnn_bbox", "mrcnn_mask"])
elif init_with == "custom":
    weights_path = os.path.join(MODEL_DIR, custom_weights_file)
    print("Loading {} as weights file".format(weights_path))
    model.load_weights(weights_path, by_name=True, exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", "mrcnn_bbox", "mrcnn_mask"])

## Training

### Starting to train
Train in two stages:
1. Only the heads. Here we're freezing all the backbone layers and training only the randomly initialized layers (i.e. the ones that we didn't use pre-trained weights from MS COCO). To train only the head layers, pass `layers='heads'` to the `train()` function.

2. Fine-tune all layers. For this simple example it's not necessary, but we're including it to show the process. Simply pass `layers="all` to train all layers.

In [ ]:
if TOAST:
    toaster.show_toast("Mask R-CNN Training Tool",
                       f"Starting training ({NB_STEPS} steps, {NB_EPOCHS} epochs)",
                       icon_path=None,
                       duration=10,
                       threaded=True)
start_time = time()

In [ ]:
import imgaug.augmenters as iaa
seq = iaa.Sequential([
    iaa.Fliplr(0.5),
    iaa.Flipud(0.5)
])

In [ ]:
# Train the head branches
# Passing layers="heads" freezes all layers except the head
# layers. You can also pass a regular expression to select
# which layers to train by name pattern.
model.train(dataset_train, dataset_val,
            learning_rate=config.LEARNING_RATE, 
            epochs=NB_EPOCHS, 
            layers='heads',
            augmentation=seq)

In [ ]:
# Fine tune all layers
# Passing layers="all" trains all layers. You can also 
# pass a regular expression to select which layers to
# train by name pattern.
# model.train(dataset_train, dataset_val, learning_rate=config.LEARNING_RATE / 10, epochs=2, layers="all")

In [ ]:
training_time = round(time() - start_time)
if TOAST:
    toaster.show_toast("Mask R-CNN Training Tool",
                       f"Finish training ({NB_STEPS} steps, {NB_EPOCHS} epochs) in {formatTime(training_time)} (avg {formatTime(round(training_time / NB_EPOCHS))}",
                       icon_path=None,
                       duration=10,
                       threaded=True)

In [ ]:
# Save weights
# Typically not needed because callbacks save after every epoch
# Uncomment to save manually
weights_saved_file = f"mask_rcnn_nephrologie{'_cortex' if cortexMode else ''}_{NB_STEPS}_{NB_EPOCHS}.h5"
model_path = os.path.join(MODEL_DIR, weights_saved_file)
model.keras_model.save_weights(model_path)

## Detection

### Initialisation of the inference model and loading of weights 

In [ ]:
class InferenceConfig(CustomConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

inference_config = InferenceConfig()

In [ ]:
# Recreate the model in inference mode
model = modellib.MaskRCNN(mode="inference", config=inference_config, model_dir=MODEL_DIR)

# Get path to saved weights
# Either set a specific path or find last trained weights
# model_path = os.path.join(ROOT_DIR, ".h5 file name here")
model_path = model.find_last()[1]
if model_path is None or not os.path.exists(model_path):
    model_path = os.path.join(MODEL_DIR, weights_saved_file)

# Load trained weights (fill in path to trained weights here)
assert model_path != "", "Provide path to trained weights"
print("Loading weights from ", model_path)
model.load_weights(model_path, by_name=True)

### Inference on random selection

Display of randomly chosen image with exepected result

In [ ]:
# Test on a random image
image_id = random.choice(dataset_val.image_ids)
original_image, image_meta, gt_class_id, gt_bbox, gt_mask = modellib.load_image_gt(dataset_val, inference_config,
                                                                                   image_id, use_mini_mask=False)

log("original_image", original_image)
log("image_meta", image_meta)
log("gt_class_id", gt_class_id)
log("gt_bbox", gt_bbox)
log("gt_mask", gt_mask)

_ = visualize.display_instances(original_image, gt_bbox, gt_mask, gt_class_id, dataset_train.class_names, 
                                colorPerClass=COLOR_PER_CLASS, figsize=(16, 16), 
                                title=f"{map_ids[image_id]} Expected",
                                fileName=f"logs/images/[{NB_STEPS}_{NB_EPOCHS}] Expected")

Detection and display of the predicted image

In [ ]:
results = model.detect([original_image], verbose=1)

r = results[0]
fig_, ax_ = get_ax(size=16)
_ = visualize.display_instances(original_image, r['rois'], r['masks'], r['class_ids'], dataset_val.class_names, 
                                r['scores'], colorPerClass=COLOR_PER_CLASS, figsize=(16, 16), ax=ax_, fig=fig_, 
                                title=f"{map_ids[image_id]} Predicted",
                                fileName=f"logs/images/[{NB_STEPS}_{NB_EPOCHS}] Predicted")

## Evaluation

### Mean Average Precision (mAP) computation on a multiple-detection test 

In [ ]:
# Compute VOC-Style mAP @ IoU=0.5
#Running on 30 images by default. Increase for better accuracy.
# image_ids = np.random.choice(dataset_val.image_ids, evaluation_size)
image_ids = dataset_val.image_ids
random.shuffle(image_ids)
APs = []
confusion_matrix = np.zeros((NB_CLASS + 1, NB_CLASS + 1), dtype=np.int32)
map_time = time()
if TOAST:
    toaster.show_toast("Mask R-CNN Training Tool",
                       "Starting mAP computation",
                       icon_path=None,
                       duration=10,
                       threaded=True)
progressBar(0, len(image_ids), "Computing mAP : ")
for idx, image_id in enumerate(image_ids):
    if TOAST and idx + 1 == evaluation_size // 2:
        toaster.show_toast("Mask R-CNN Training Tool", "50% mAP computation achieved",
                           icon_path=None, duration=10, threaded=True)
    # Load image and ground truth data
    image, image_meta, gt_class_id, gt_bbox, gt_mask =\
        modellib.load_image_gt(dataset_val, inference_config, image_id, use_mini_mask=False)
    molded_images = np.expand_dims(modellib.mold_image(image, inference_config), 0)
    # Run object detection
    results = model.detect([image], verbose=0)
    r = results[0]
    # Compute AP
    AP, precisions, recalls, overlaps, temp_confusion_matrix = \
            utils.compute_ap(gt_bbox, gt_class_id, gt_mask, r["rois"],
                             r["class_ids"], r["scores"], r['masks'],
                             nb_class=NB_CLASS, 
                             confusion_iou_threshold=0.1)
    if idx + 1 == evaluation_size:
        map_time = round(time() - map_time)
    progressBar(idx + 1, evaluation_size, "Computing mAP : ", suffix=f" ({formatTime(map_time)})")
    APs.append(AP)
    confusion_matrix = np.add(confusion_matrix, temp_confusion_matrix)

mAP = np.mean(APs)
mAP_str = str(mAP).replace('.', ',')
print(f"mAP: {mAP:06.2%}")
name = f"[{NB_STEPS}_{NB_EPOCHS}] Confusion Matrix"
name2 = f"[{NB_STEPS}_{NB_EPOCHS}] Normalized Confusion Matrix"
cmap = plt.cm.get_cmap('hot')
visualize.display_confusion_matrix(confusion_matrix, 
                                   dataset_val.get_class_names(), 
                                   title=name, cmap=cmap, show=False,
                                   fileName=os.path.join(CONF_MATRIX_DIR, name))
visualize.display_confusion_matrix(confusion_matrix, 
                                   dataset_val.get_class_names(), 
                                   title=name2, cmap=cmap, show=False, 
                                   normalize=True, 
                                   fileName=os.path.join(CONF_MATRIX_DIR, name2))
with open(RESULT_CSV_PATH, "a") as file:
    file.write(f'"{ctime(time())}"; "{init_with}"; "{weights_saved_file}"; {NB_STEPS}; {NB_EPOCHS}; {training_time}; {mAP_str}\n')

In [ ]:
if TOAST:
    toaster.show_toast("Mask R-CNN Training Tool",
                       f"mAP computed : {mAP:06.2%}%\nDuration : {formatTime(map_time)}",
                       icon_path=None,
                       duration=30,
                       threaded=True)